### 자동 미분 (torch.autograd)

* Pytorch의 autograd는 신경망 훈련을 지원하는 자동 미분 기능
* torch.autograd 동작 방법

  * 텐서에 .requires_grad 속성을 True로 설정하면, 이후의 텐서 모든 연산들을 추적함
  * 텐서 .backward()를 호출하면, 연산에 연결된 각 텐서들의 미분 값을 계산하여, 각 텐서 객체.grad에 저장

    * .requires_grad_()는 연결된 Tensor로부터의 계산된 자동미분 값을, 다시 현 텐서부터 시작하도록 만듬

### 신경망 동작 이해

* 모델 및 데이터 생성
* forward pass로 입력 데이터를 모델에 넣어서 예측값 계산
* 예측값과 실제값의 차이를 loss function으로 계산
* backward pass 로 각 모델 파라미터를 loss 값 기반 미분하여 저장
* optimizer로 모델 파라미터의 최적값을 찾기 위해, 파라미터 값 업데이트


### 코드로 이해하는 autograd

* 텐서에 .requires_grad 속성을 True로 설정
* .requires_grad 속성이 True로 설정되면, 텐서의 모든 연산 추적을 위해, 내부적으로 방향성 비순환 그래프(DAG: Directed Acyclic Graph)를 동적 구성

  * 방향성 비순환 그래프(DAG)의 leave 노드는 입력 텐서이고, root 노드는 결과 텐서가 됨
  

In [1]:
import torch

x = torch.rand(1, requires_grad=True)
y = torch.rand(1)
y.requires_grad=True
loss = y - x

In [2]:
loss.backward()
print(x.grad, y.grad)

tensor([-1.]) tensor([1.])


### 코드로 이해하는 autograd

* 입력 차원이 4, 출력 차원이 3이고, Linear Layer 함수를 $f(x) = x * W + b$으로 정의
* requires_grad = True 인 텐서와 연결되어 계산되는 텐서는 grad_fn 속성을 가지며, 관련 정보는 backward()시 사용됨

In [18]:
x = torch.ones(4) # input tensor
y = torch.zeros(3) # expected output
W = torch.rand(4, 3, requires_grad=True)
b = torch.rand(3, requires_grad=True)
z = torch.matmul(x, W) + b
print(W, b, z)

tensor([[0.2964, 0.4378, 0.6806],
        [0.6602, 0.9106, 0.4719],
        [0.2707, 0.8291, 0.5090],
        [0.5135, 0.4646, 0.5162]], requires_grad=True) tensor([0.2041, 0.9772, 0.9917], requires_grad=True) tensor([1.9449, 3.6192, 3.1694], grad_fn=<AddBackward0>)


### Loss 함수와 MSE

* MSE(Mean Squared Error) : 평균 제곱근 오차

  * 실제 값과 예측 값의 차이를 제곱한 값의 평균


In [19]:
import torch.nn.functional as F

loss = F.mse_loss(z, y)
loss.backward()
print(loss, W.grad, b.grad)

tensor(8.9756, grad_fn=<MseLossBackward0>) tensor([[1.2966, 2.4128, 2.1129],
        [1.2966, 2.4128, 2.1129],
        [1.2966, 2.4128, 2.1129],
        [1.2966, 2.4128, 2.1129]]) tensor([1.2966, 2.4128, 2.1129])


### 파라미터 업데이트

In [24]:
threshold = 0.1
learning_rate = 0.01
iteration_num = 0

while loss > threshold:
  iteration_num += 1
  W = W - learning_rate * W.grad
  b = b - learning_rate * b.grad
  print(iteration_num, loss, z , y)

  # detach_() : 텐서를 기존 방향성 비순환 그래프 (DAG : Directed Acyclic Graph)로 부터 끊음
  # .requires_grad_(True) : 연결된 Tensor로부터의 계산된 자동미분 값을, 다시 현 텐서부터 시작하도록 만듬
  W.detach_().requires_grad_(True)
  b.detach_().requires_grad_(True)

  z = torch.matmul(x, W) + b
  loss = F.mse_loss(z, y)
  loss.backward()

print("최종결과:", iteration_num + 1, loss, z , y)

최종결과: 1 tensor(0.0955, grad_fn=<MseLossBackward0>) tensor([0.2007, 0.3734, 0.3270], grad_fn=<AddBackward0>) tensor([0., 0., 0.])
